This is a simple example showing how to use this package to train a machine learning force field using your data.

In this example, we will study a system of two helium atoms interacting through a Lennard-Jones potential. The example is divided into two main sections:

1) Model Setup and Training:
    We will load a pre-generated dataset containing configurations of two helium atoms in a simulation box and use the SymmLearn functions to build and train our neural network model.

2) Results and Comparison:
    Finally, we will analyze the results. For this simple case, it is possible to visualize the trained force field and compare it to the reference Lennard-Jones potential.


In [1]:
#using Symmlearn
using Plots

include("../src/Data_prep.jl")
include("../src/Utils.jl")
include("../src/Train.jl")
include("../src/Model.jl")


dispatch (generic function with 2 methods)

The loading process of the .xyz dataset con be done as illustrated here in the next code block.

Everything is fuly explained in the documentation, it's important to mention that the target data is stored in a Vector of Sample structs, containing both the sample energy and the forces

This dataset consists in 500 samples,for each of them the energy was computed using a Lennard-Jones potential with $\sigma$ = 1 and $\epsilon$ = 1.

For each sample the distance between the two Helium atoms was randomly generated between 0.95 $\sigma$ and 2.5 $\sigma$




In [2]:
file_path = "helium_LJ_dataset.xyz"

Train, Val, Test_data, energy_mean, energy_std, _, _, unique_species, species_idx, _= xyz_to_nn_input(file_path)



println(" Now the data is in the correct format")

 Now the data is in the correct format


The xyz_to_nn_input function returns the data already split in test, train and validation, the mean and the standard deviation of both the energies and the forces in order to renormalize them later and the lattice parameters, used by the model to compute the atomic distances with periodic boundary conditions ( in this example we won't be using PBC as the helium atoms are confined in a box )

In the next block we wil building and check if our model behaves as expected,  since the system is trivial using the forces to train it won't be needed

In [3]:
#define the model using 5 G1 symmetry functions


model = build_species_models(unique_species, species_idx, 5, 2.5f0)

#We can check if the model and the loss work as we expected on a small batch 

x_batch = Train[1][1:3, :]
y_batch = Train[2][1:3]

e = extract_energies(y_batch)
f = 0f0 # we won't need to train the model using the forces



println("Model output with a batch as input: ", dispatch(x_batch , model))
println("Model loss with batch input: ", loss(model, x_batch, e , f))




Model output with a batch as input: Float32[-0.07396797; -0.064360924; -0.12967743;;]
Model loss with batch input: 1.8667735


And now we can train it

In [4]:
#we train the model setting the parameter forces to false

last_model, best_model, train_loss, val_loss = train_model!(
        model,
        Train[1], 
        Train[2], 
        Val[1],
        Val[2],
        loss;
         forces = false , initial_lr = 0.1, epochs = 1000, batch_size = 32
    )

Progress: 100%|█████████████████████████████████████████| Time: 0:03:06mm


(Chain[Chain(G1Layer(Float32[2.5679374, 2.7176106, 2.543765, 1.4128323, 1.1646677], Float32[2.5874572, 4.3358736, 2.4181352, 0.8371488, 0.86098343], 2.5f0, 2.0f0), Dense(5 => 16, tanh), Dense(16 => 8, tanh), Dense(8 => 4, tanh), Dense(4 => 2, tanh), Dense(2 => 1))], Chain[Chain(G1Layer(Float32[1.8561207, 1.0738552, 2.9820657, 2.118928, 1.2720356], Float32[1.8004206, 2.4452336, 0.33879507, -0.18586907, 1.6711178], 2.5f0, 2.0f0), Dense(5 => 16, tanh), Dense(16 => 8, tanh), Dense(8 => 4, tanh), Dense(4 => 2, tanh), Dense(2 => 1))], Float32[301.71658, 310.81165, 321.24463, 298.6533, 441.556, 321.92975, 323.66394, 327.01126, 332.15744, 305.40823  …  217.32259, 217.32256, 217.32253, 217.3225, 217.32246, 217.32248, 217.32245, 217.32243, 217.32237, 217.32236], Float32[134.4477, 146.21384, 152.23833, 138.73094, 162.06305, 152.10323, 153.00461, 135.99384, 157.27815, 133.95978  …  195.29173, 195.2888, 195.29013, 195.27377, 195.29291, 195.27274, 195.28069, 195.28104, 195.28404, 195.30612])

Now our model has been trained, we can look at the results, the plot compares the energy of each pair as a function of the distance between the two atoms with the LJ potential for the test set

In [ ]:
# Denormalize model predictions for the test set using Z-score inversion
test_guess = dispatch(Test_data[1], best_model) .* energy_std .+ energy_mean


# Extract interatomic distances from the test set (for plotting)
test_distances = [Test_data[1][i][2].coord[1] for i in 1:size(Test_data[1], 1)]


# Define Lennard-Jones potential curve for reference
r = 0.95:0.01:2.5
lj_energy =  ((1 ./ r).^12 .- (1 ./ r).^6)

plot(r, lj_energy,
    label="Lennard-Jones Potential",
    color="black",
    lw=2
)

# Plot predictions vs Lennard-Jones potential
scatter!(test_distances, test_guess,
    label="Model prediction (Test Set)",
    alpha=0.5,
    color="cyan"
)



xlabel!("Distance [σ]")
ylabel!("Energy [ϵ]")
title!("Helium energy prediction - Test Set")


-0.05487593


LoadError: Cannot convert Float32 to series data for plotting

As expected the model managed to reproduce very well the Lennard Jones potential